In [1]:
import os
import string

import networkx as nx
import nltk
from nltk.util import ngrams

In [2]:
graph = nx.read_gexf('out/elbulli_dat.gexf')

In [3]:
g_nlg = nx.read_gexf('out/elbulli_nlg.gexf')

In [4]:
all_ingredients_graph = nx.read_gexf('data/spanish_ingredients_lexicon_5.gexf')

In [5]:
ingredients = all_ingredients_graph.nodes()

In [6]:
len(ingredients)

26743

In [7]:
ingredients[:10]

['polvos para pudines',
 'zumo melon',
 'aguas de sardinas',
 'hongo algas hongos nori sequías',
 'gallineta',
 'salmón ahumados',
 'pures de arándanos',
 'salsas de tomates fritos',
 'semillas de alegria gellan',
 'infusiones anisados']

In [8]:
all_techniques_graph = nx.read_gexf('data/spanish_techniques_lexicon_5.gexf')

In [9]:
techniques = all_techniques_graph.nodes()

In [10]:
len(techniques)

4372

In [11]:
techniques[:10]

['caldea blanco',
 'dorando a presionando',
 'cocina vacío',
 'fria a presión',
 'cocción presione',
 'frito presionar',
 'freir a presione',
 'rotamos a presiona',
 'fria presión',
 'al grillando presione']

In [12]:
for n, data in g_nlg.nodes(data=True):
    if data['nodetype'] == 'Producto':
        print(n)
        print(data)
        break
for n, data in g_nlg.nodes(data=True):
    if data['nodetype'] == 'sabor':
        print(n)
        print(data)
        break

Producto:láminas de falso tartufo
{'label': 'Producto:láminas de falso tartufo', 'nodetype': 'Producto'}
sabor:queso Idiazábal ahumado
{'label': 'sabor:queso Idiazábal ahumado', 'nodetype': 'sabor'}


In [13]:
def my_split(s):
    if '#' in s:
        r = s.split('#')
    else:
        r = s.split('<br>')
    return r

def my_trim(s):
    return ' '.join(s.split())

def my_ngrams(technique):
    ngrms = []
    tokens = nltk.word_tokenize(technique)
    for i in range(1, len(tokens) + 1):
        ngrms.extend(ngrams(tokens, i))
    return list(map(lambda x: ' '.join(x), ngrms))

def in_any(e, es):
    return any(map(lambda x: e in x, es))

In [15]:
%%time

for n, data in graph.nodes(data=True):
    if data['nodetype'] == 'Elaboracion':
        ingrs_str = data['ingrs']
        ingreds = set()
        elems = my_split(ingrs_str)
        ingrs = map(my_trim, elems)
        for ingr in ingrs:
            ngrms = my_ngrams(ingr)
            ngrms.reverse()
            for ngrm in ngrms:
                if ngrm in ingredients:
                    v = ngrm
                    graph.add_node(v, {'label': v, 'nodetype': 'ingrediente'})
                    graph.add_edge(n, v, {'edgetype': 'composicion'})
                    break # I assume one ingredient only per split
#         del(data['ingrs'])
        desc = data['desc']
        techns = set()
        elems = my_split(desc)
        steps = map(my_trim, elems)
        for step in steps:
            used_ngrams = set()
            ngrms = my_ngrams(step)
            ngrms.reverse()
            for ngrm in ngrms:
                if ngrm in techniques and not in_any(ngrm, used_ngrams):
                    v = ngrm
                    graph.add_node(v, {'label': v, 'nodetype': 'tecnica'})
                    graph.add_edge(n, v, {'edgetype': 'tecnica'})
                    used_ngrams.add(ngrm) # I assume one or more techniques per split
#         del(data['desc'])

KeyboardInterrupt: 

In [16]:
prep_prod_rels = ['bañado', 'alcohol', 'chocolate', 'lacteo', 'nuevaPasta', 'producto', 'relleno']
prep_ingr_rels = ['composicion']
prep_flav_rels = ['sabor']

def get_prep_products(g, prep):
    return {k for k in g[prep] if g[prep][k]['edgetype'] in prep_prod_rels}

def get_prep_ingredients(g, prep):
    return {k for k in g[prep] if g[prep][k]['edgetype'] in prep_ingr_rels}

def get_prep_flavors(g, prep):
    return {k for k in g[prep] if g[prep][k]['edgetype'] in prep_flav_rels}

def get_prep_components(g, prep):
    prods = get_prep_products(g, prep)
    ingrs = get_prep_ingredients(g, prep)
    flavs = get_prep_flavors(g, prep)
    return prods.union(ingrs).union(flavs)

def get_recip_preparations(g, recip):
    return [k for k in g[recip] if g[recip][k]['edgetype'] == 'elaboracion']

def get_recip_products(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_products(g, prep)}

def get_recip_ingredients(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_ingredients(g, prep)}

def get_recip_flavors(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_flavors(g, prep)}

def get_recip_components(g, recip):
    prods = get_recip_products(g, recip)
    ingrs = get_recip_ingredients(g, recip)
    flavs = get_recip_flavors(g, recip)
    return prods.union(ingrs).union(flavs)

In [ ]:
ddd={'a':0,'b':0,'c':0}

def equals_preparations(n1, n2):
    ddd['a']+=1
    r = False
    data1 = graph.node[n1]
    data2 = graph.node[n2]
    if data1['title'] == data2['title']:
        comps1 = get_prep_components(graph, n1)
        comps2 = get_prep_components(graph, n2)
        if comps1 == comps2:
            r = True
            ddd['b']+=1
        else:
            print(data1['title'], data2['title'])
            ddd['c']+=1
    return r

In [ ]:
%%time

preps = set()
for n1, data1 in graph.nodes(data=True):
    if data1['nodetype'] == 'Elaboracion':
        for n2, data2 in graph.nodes(data=True):
            if data2['nodetype'] == 'Elaboracion':
                if n1 != n2:
                    if equals_preparations(n1, n2):
                        preps.add(tuple(sorted([n1, n2])))

Para las cebollas tiernas Para las cebollas tiernas
Para las cebollas tiernas Para las cebollas tiernas
Para la gelatina de albahaca Para la gelatina de albahaca
Para la gelatina de albahaca Para la gelatina de albahaca
Para la gelatina de albahaca Para la gelatina de albahaca
Para la gelatina de albahaca Para la gelatina de albahaca
Para la gelatina de albahaca Para la gelatina de albahaca
Para la gelatina de albahaca Para la gelatina de albahaca
Para el zumo de manzana Para el zumo de manzana
Para el zumo de manzana Para el zumo de manzana
Para el zumo de manzana Para el zumo de manzana
Para el zumo de manzana Para el zumo de manzana
Para el zumo de manzana Para el zumo de manzana
Para el zumo de manzana Para el zumo de manzana
Para el zumo de manzana Para el zumo de manzana
Para el zumo de manzana Para el zumo de manzana
Para el zumo de manzana Para el zumo de manzana
Para el zumo de manzana Para el zumo de manzana
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Ot

Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otro

Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otro

Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otro

Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otro

Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Para el consomé Para el consomé
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otro

Para el aceite de piña verde de pino Para el aceite de piña verde de pino
Para el aceite de piña verde de pino Para el aceite de piña verde de pino
Para el aceite de piña verde de pino Para el aceite de piña verde de pino
Para el aceite de piña verde de pino Para el aceite de piña verde de pino
Para el aceite de piña verde de pino Para el aceite de piña verde de pino
Para el aceite de piña verde de pino Para el aceite de piña verde de pino
Para el aceite de piña verde de pino Para el aceite de piña verde de pino
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Ot

Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Otros Otros
Para el jarabe base Para el jarabe base
Para el jarabe base Para el jarabe base
Para el jarabe base Para el jarabe base
Para el jarabe base Para el jarabe base
Para el jara

In [ ]:
ddd

In [68]:
len(preps)

453567

In [20]:
# es_test = [
#     (1,2),(1,3),(2,3),(3,4),
#     (5,6),(6,7),(7,8),(7,9),
#     (10,11),(11,12),(12,13),
# #     (1,13),
#     (6,12),(1, 14)
# ]

In [69]:
%%time

ls=[]
for e1, e2 in es:
    found = False
    first_set = None
    to_remove=[]
    for x in ls:
        if (e1 in x or e2 in x) and not found:
            x.add(e1)
            x.add(e2)
            found = True
            first_set = x
        if x != first_set and (e1 in x or e2 in x) and found:
            x.add(e1)
            x.add(e2)
            for y in x:
                first_set.add(y)
            to_remove.append(x)
    if not found:
        ls.append({e1, e2})
    for y in to_remove:
        ls.remove(y)

In [70]:
len(ls)

821

In [71]:
for x in ls:
    ts = set()
    ingrs = set()
    descs = set()
    for y in x:
        ts.add(graph.node[y]['title'])
        ingrs.add(graph.node[y]['ingrs'])
        descs.add(graph.node[y]['desc'])
    if len(ts) != 1:
        print('AAA')
    break
#     if len(ingrs) != 1:
#         print('BBB')
#     if len(descs) != 1:
#         print('CCC')

In [72]:
list(x)[:4]

['799-8', '1003-5', '999-6', '262-9']

In [73]:
for y in list(ls[0])[:4]:
    print(graph.node[y]['title'])
    print(graph.node[y]['ingrs'])
    print(graph.node[y]['desc'])

Otros
          sal Maldon#          cacao en un espolvoreador#          la piel rallada de una mandarina#          la piel rallada de una lima#          la piel rallada de una naranja

Otros
     10 g de vinagre balsámico tradicional #          introducido en pipetas

Otros
   120 hojas de albahaca fresca de 0,5 cm #          de longitud#   100 g de azúcar

Otros
        4 brotes de romero fresco
######


In [74]:
equals_preparations('799-8', '1003-5')

True

In [75]:
get_prep_components(graph, '799-8')

set()

In [77]:
get_prep_products(graph, '799-8')

set()

In [81]:
{k for k in graph['799-8'] if graph['799-8'][k]['edgetype'] in prep_prod_rels}

set()

In [82]:
for k in graph['799-8']:
    print(k)

In [83]:
graph.node['799-8']

{'desc': '',
 'ingrs': '          sal Maldon#          cacao en un espolvoreador#          la piel rallada de una mandarina#          la piel rallada de una lima#          la piel rallada de una naranja',
 'label': '799-8',
 'nodetype': 'Elaboracion',
 'title': 'Otros'}

In [78]:
get_prep_ingredients(graph, '799-8')

set()

In [79]:
get_prep_flavors(graph, '799-8')

set()

In [76]:
get_prep_components(graph, '1003-5')

set()

In [68]:
c=d=0
for n, data in g_nlg.nodes(data=True):
    if data['nodetype'] == 'Elaboracion':
        c+=1
for n, data in graph.nodes(data=True):
    if data['nodetype'] == 'Elaboracion':
        d+=1

In [69]:
c

4636

In [70]:
d

7052